In [1]:
import numpy as np
import json
import os
from python_tsp.heuristics import solve_tsp_local_search, solve_tsp_simulated_annealing
from python_tsp.exact import solve_tsp_dynamic_programming
from time import time
from os.path import join as pjoin
from tivlib import TIV
from utils.ph_harm import *
from tqdm import tqdm
import random
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials


In [2]:
# we load harmonic features and generate a playlist from it
hpcp = np.load('spoty_hpcps.npy')
with open('spoty_hfeats.json', 'r') as f:
    HFEATS = json.load(f)
for i, item in enumerate(HFEATS):
    HFEATS[item]['hpcp'] = hpcp[i]  

with open('top1000_playlists.json', 'r') as f:
    playlists = json.load(f)

# Harmonic compatibility distances

In [3]:
# Valid transitions in the circle of fifths (plus the enharmonics):
CIRCLE_TRANSITIONS = {'Cmajor': ['Cmajor', 'Aminor', 'Fmajor', 'Gmajor'],
               'Gmajor': ['Gmajor', 'Dmajor', 'Cmajor', 'Eminor'],               
               'Dmajor': ['Dmajor', 'Amajor', 'Gmajor', 'Bminor'],               
               'Amajor': ['Amajor', 'Dmajor', 'Emajor', 'F#minor'],              
               'Emajor': ['Emajor', 'Amajor', 'Bmajor', 'Cbmajor', 'C#minor'],            
               'Bmajor': ['Bmajor', 'Cbmajor', 'Emajor', 'F#major', 'Gbmajor', 'G#minor', 'Abminor'],
               'Cbmajor': ['Bmajor', 'Cbmajor', 'Emajor', 'F#major', 'Gbmajor', 'G#minor', 'Abminor'],
               'F#major': ['F#major', 'Gbmajor', 'Bmajor', 'Cbmajor', 'Dbmajor', 'C#major', 'D#minor', 'Ebminor'],
               'Gbmajor': ['Gbmajor', 'F#major', 'Bmajor', 'Cbmajor', 'Dbmajor', 'C#major', 'D#minor', 'Ebminor'],               
               'C#major': ['C#major', 'Dbmajor', 'F#major', 'Gbmajor', 'Abmajor', 'Bbminor', 'A#minor'],
               'Dbmajor': ['Dbmajor', 'C#major', 'F#major', 'Gbmajor', 'Abmajor', 'Bbminor', 'A#minor'],             
               'Abmajor': ['Abmajor', 'Dbmajor', 'C#major', 'Ebmajor', 'Fminor'],               
               'Ebmajor': ['Ebmajor', 'Abmajor', 'Bbmajor', 'Cminor'],
               'Bbmajor': ['Bbmajor', 'Ebmajor', 'Fmajor', 'Gminor'],  
               'Fmajor': ['Fmajor', 'Bbmajor', 'Cmajor', 'Dminor'],               
               'Aminor': ['Aminor', 'Eminor', 'Dminor', 'Cmajor'],               
               'Eminor': ['Eminor', 'Bminor', 'Aminor', 'Gmajor'],              
               'Bminor': ['Bminor', 'F#minor', 'Eminor', 'Dmajor'],               
               'F#minor': ['F#minor', 'C#minor', 'Bminor', 'Amajor'],               
               'C#minor': ['C#minor', 'F#minor', 'Abminor', 'G#minor', 'Emajor'],               
               'G#minor': ['G#minor', 'Abminor', 'C#minor', 'Ebminor', 'D#minor', 'Bmajor', 'Cbmajor'],
               'Abminor': ['Abminor', 'G#minor', 'C#minor', 'Ebminor', 'D#minor', 'Bmajor', 'Cbmajor'],              
               'D#minor': ['D#minor', 'Ebminor', 'Abminor', 'G#minor', 'Bbminor', 'A#minor', 'F#major', 'Gbmajor'],
               'Ebminor': ['Ebminor', 'D#minor', 'Abminor', 'G#minor', 'Bbminor', 'A#minor', 'F#major', 'Gbmajor'],               
               'A#minor': ['A#minor', 'Bbminor', 'Ebminor', 'D#minor', 'Fminor', 'Dbmajor', 'C#major'],
               'Bbminor': ['Bbminor', 'A#minor', 'Ebminor', 'D#minor', 'Fminor', 'Dbmajor', 'C#major'],               
               'Dminor': ['Dminor', 'Aminor', 'Gminor', 'Fmajor'],               
               'Gminor': ['Gminor', 'Dminor', 'Cminor', 'Bbmajor'],               
               'Cminor': ['Cminor', 'Gminor', 'Fminor', 'Ebmajor'],
               'Fminor': ['Fminor', 'Bbminor', 'A#minor', 'Cminor', 'Abmajor']}

In [4]:
# coordinates on the circle of fifths:
maj_keys = ['Cmajor', 'Gmajor', 'Dmajor', 'Amajor', 'Emajor', 'Bmajor', 'F#major', 'C#major', 'Abmajor',
 'Ebmajor', 'Bbmajor', 'Fmajor']
min_keys = ['Aminor', 'Eminor', 'Bminor', 'F#minor', 'C#minor', 'G#minor', 'D#minor', 'A#minor', 'Fminor',
 'Cminor', 'Gminor', 'Dminor']

l = 3.863708 # radius of the circle. this value makes vertical and horizontal translation the same
l = l*2
h = 2. # distance between adjacent keys
h = h * 2
CIRCLE_COORDINATE = {}
for i, key in enumerate(maj_keys):
    x = l * np.cos((np.pi * i) / 6)
    y = l * np.sin((np.pi * i) / 6)
    z = 0.
    CIRCLE_COORDINATE[key] = np.array([x,y,z])
for i, key in enumerate(min_keys):
    x = l * np.cos((np.pi * i) / 6)
    y = l * np.sin((np.pi * i) / 6)
    z = h
    CIRCLE_COORDINATE[key] = np.array([x,y,z])   
#and we add the enharmonic equivalents
CIRCLE_COORDINATE['Cbmajor'] = CIRCLE_COORDINATE['Bmajor']
CIRCLE_COORDINATE['Gbmajor'] = CIRCLE_COORDINATE['F#major']
CIRCLE_COORDINATE['Dbmajor'] = CIRCLE_COORDINATE['C#major']
CIRCLE_COORDINATE['Abminor'] = CIRCLE_COORDINATE['G#minor']
CIRCLE_COORDINATE['Ebminor'] = CIRCLE_COORDINATE['D#minor']
CIRCLE_COORDINATE['Bbminor'] = CIRCLE_COORDINATE['A#minor']

# Filling distance matrixes as required by python_tsp

### Diversity metrics (not used in the evaluation)

In [5]:
def diver_binary(playlist):
    distance_matrix = np.full((len(playlist['tracks']), len(playlist['tracks'])), None)
    for i, current_song in enumerate(playlist['tracks']):
        for j, next_song in enumerate(playlist['tracks']):
            if distance_matrix[i,j] is None:
                current_uri = playlist['tracks'][i]['track_uri']
                next_uri = playlist['tracks'][j]['track_uri']
                current_key = HFEATS[current_uri]['key']
                next_key = HFEATS[next_uri]['key']
                if next_key != current_key:
                    distance_matrix[i,j] = 0
                else:
                    distance_matrix[i,j] = 1
                    distance_matrix[j,i] = distance_matrix[i,j]
    distance_matrix[:, 0] = 0
    return distance_matrix

In [6]:
def cosine_similarity(x, y):
    return np.dot(x, y) / (np.sqrt(np.dot(x, x)) * np.sqrt(np.dot(y, y)))

In [7]:
def diver_hpcp(playlist):
    distance_matrix = np.full((len(playlist['tracks']), len(playlist['tracks'])), None)
    for i, current_song in enumerate(playlist['tracks']):
        for j, next_song in enumerate(playlist['tracks']):
            if distance_matrix[i,j] is None:
                current_uri = playlist['tracks'][i]['track_uri']
                next_uri = playlist['tracks'][j]['track_uri']
                current_hpcp = HFEATS[current_uri]['hpcp']
                next_hpcp = HFEATS[next_uri]['hpcp']
                distance_matrix[i,j] = cosine_similarity(current_hpcp, next_hpcp)
                distance_matrix[j,i] = distance_matrix[i,j] 
    distance_matrix[:, 0] = 0
    return distance_matrix

### Compatibility distances

In [8]:
def comp_binary(playlist):
    #Binary method.
    #Adjacent boxes in the circle of fiths transitions have 0 cost, 1 otherwise
    distance_matrix = np.full((len(playlist['tracks']), len(playlist['tracks'])), None)
    for i, current_song in enumerate(playlist['tracks']):
        for j, next_song in enumerate(playlist['tracks']):
            if distance_matrix[i,j] is None:
                current_uri = playlist['tracks'][i]['track_uri']
                next_uri = playlist['tracks'][j]['track_uri']
                current_key = HFEATS[current_uri]['key']
                next_key = HFEATS[next_uri]['key']
                if next_key in CIRCLE_TRANSITIONS[current_key]:
                    distance_matrix[i,j] = 0
                else:
                    distance_matrix[i,j] = 1
                distance_matrix[j,i] = distance_matrix[i,j]  
    distance_matrix[:, 0] = 0
    return distance_matrix

In [9]:
def comp_circle(playlist):
    #Circle method
    #Euclidean distance on the coordinates of the circle of fifths in R3
    distance_matrix = np.full((len(playlist['tracks']), len(playlist['tracks'])), None)
    for i, current_song in enumerate(playlist['tracks']):
        for j, next_song in enumerate(playlist['tracks']):
            if distance_matrix[i,j] is None:
                current_uri = playlist['tracks'][i]['track_uri']
                next_uri = playlist['tracks'][j]['track_uri']
                current_key = HFEATS[current_uri]['key']
                next_key = HFEATS[next_uri]['key']
                current_coordinate = CIRCLE_COORDINATE[current_key]
                next_coordinate = CIRCLE_COORDINATE[next_key]
                distance_matrix[i,j] = np.linalg.norm(current_coordinate-next_coordinate)
                distance_matrix[j,i] = distance_matrix[i,j]
    distance_matrix[:, 0] = 0
    return distance_matrix

In [10]:
def comp_TIV(playlist):
    # TIV method
    # use TIVlib small_scale_compatibility
    distance_matrix = np.full((len(playlist['tracks']), len(playlist['tracks'])), None)
    for i, current_song in enumerate(playlist['tracks']):
        for j, next_song in enumerate(playlist['tracks']):
            if distance_matrix[i,j] is None:
                current_uri = playlist['tracks'][i]['track_uri']
                next_uri = playlist['tracks'][j]['track_uri']
                current_hpcp = HFEATS[current_uri]['hpcp']
                next_hpcp = HFEATS[next_uri]['hpcp']
                distance_matrix[i,j] = TIV.from_pcp(current_hpcp).small_scale_compatibility(TIV.from_pcp(next_hpcp))
                distance_matrix[j,i] = distance_matrix[i,j]
    distance_matrix[:, 0] = 0
    return distance_matrix

In [11]:
def comp_ph(playlist):
    #Harrison & Pearce consonance
    distance_matrix = np.full((len(playlist['tracks']), len(playlist['tracks'])), None)
    for i, current_song in enumerate(playlist['tracks']):
        for j, next_song in enumerate(playlist['tracks']):
            if distance_matrix[i,j] is None:
                current_uri = playlist['tracks'][i]['track_uri']
                next_uri = playlist['tracks'][j]['track_uri']
                current_hpcp = HFEATS[current_uri]['hpcp']
                next_hpcp = HFEATS[next_uri]['hpcp']
                # We take the top-3 pitch classes from each track, rotating in order to match ph algorithm
                chord_a = (np.argsort(current_hpcp)[::-1][:3].round()+ 9) % 12
                chord_b = (np.argsort(next_hpcp)[::-1][:3].round()+ 9) % 12
                # Merge classes in a single chord
                mix = np.unique(np.append(chord_a, chord_b))
                # Compute chord harmonicity
                mlspec = milne_pc_spectrum(mix)
                distance_matrix[i,j] = ph_harmon(mlspec)
                distance_matrix[j,i] = distance_matrix[i,j] 
    distance_matrix[:, 0] = 0
    return distance_matrix

# Filter playlists for subjective evaluation

In [12]:
#Pick playlists with large key variability so as methods propose different reorderings
j=0
topn=10
playlist_selection={}
for i,playlist in enumerate(playlists):
    variability = len(list(set([HFEATS[x['track_uri']]['key'] for x in playlists[playlist]['tracks']][:topn]))) 
    if variability >= (5):
        #if i in z:
        playlist_selection[playlist] = playlists[playlist]
        playlist_selection[playlist]['tracks'] = playlist_selection[playlist]['tracks'][:topn]

In [14]:
#Authentication - without user
cid = 'b569f6b9399545fcb0b97e821ac7434f'
secret = '' # use your own credentials to Spotify API
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [15]:
#Compute optimal TSP paths with dynamic programming
result = []
for playlist in tqdm(playlist_selection):
    d = {}
    e = {}
    perm, dis = solve_tsp_dynamic_programming(comp_binary(playlist_selection[playlist]))
    d['binary'] = {'permutation': perm, 'distance': dis}
    perm, dis = solve_tsp_dynamic_programming(comp_circle(playlist_selection[playlist]))
    d['circle'] = {'permutation': perm, 'distance': dis}
    perm, dis = solve_tsp_dynamic_programming(comp_TIV(playlist_selection[playlist]))
    d['tiv'] = {'permutation': perm, 'distance': dis}
    perm, dis = solve_tsp_dynamic_programming(comp_ph(playlist_selection[playlist]))
    d['ph'] = {'permutation': perm, 'distance': dis}
    e['uris'] = [x['track_uri'] for x in playlist_selection[playlist]['tracks']]
    
    for uri in e['uris']:
        if sp.track(uri)['preview_url'] == None:
            continue
    e['playlist'] = playlist_selection[playlist]
    e['options'] = d
    result.append(e)

 42%|█████████████████▍                       | 420/991 [06:56<10:54,  1.15s/it]/home/enricguso/kakao/hr-listening-test/tivlib/tiv.py:152: RuntimeWarning: invalid value encountered in true_divide
  return TIV(self.energy+tiv2.energy, (self.energy * self.vector + tiv2.energy * tiv2.vector) / (self.energy + tiv2.energy))
100%|█████████████████████████████████████████| 991/991 [16:24<00:00,  1.01it/s]


### Curation for the subjective evaluation

In [16]:
# We filter out the playlists that have ties between methods
filtered_res = []
for i, res in enumerate(result):
    if len(list(set([str(x['permutation']) for x in list(res['options'].values())]))) == 4:
        filtered_res.append(res)

#cherry pick some of the playlists so they contain different genres:
z = [63, 66, 76, 79, 111, 147, 226, 266, 285, 458, 571, 674]
selection = []
for idx in z:
    selection.append(filtered_res[idx])

In [17]:
#Shuffle the dicts, so we shuffle the columns of the eval

for select in selection:
    select['options']['gt'] = {'permutation': [0,1,2,3,4,5,6,7,8,9], 'distance':np.nan}
    key_list = list(select['options'])
    random.shuffle(key_list)
    d2 = {}
    for key in key_list:
        d2[key]=select['options'][key]
    select['options'] = d2

In [ ]:
#save the playlists for app/main.py

with open('listening_selection_data_10.json', 'w') as f:
    json.dump(selection, f, indent=4)